In [1]:
from rdkit.Chem import *
from rdkit import *
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
import py3Dmol
mol="O=C1[C@@H](CCCCCC)CC1"#"O=C1[C@H](CC2)[C@H](CC)[C@H](CC)[C@H]2C1"#"O=C1C(C)(C2(C)(C))CC[C@@H]2C1"

In [2]:
def opt_sp_mm(mol, ff):
    # 1. 1000個のコンフォマーを発生
    #sm = Chem.MolToSmiles(mol)
    m = Chem.MolFromSmiles(mol)
    m_h = Chem.AddHs(m)
    cids = AllChem.EmbedMultipleConfs(m_h,
                                      numConfs=1000,
                                      randomSeed=1234,
                                      pruneRmsThresh=0.1,
                                      numThreads=0)

    # 2,3. 各コンフォマーを最適化し，エネルギー計算
    energy = []
    if ff == 'uff':
        for cid in cids:
            uff = AllChem.UFFGetMoleculeForceField(m_h,
                                                   confId=cid)
            uff.Minimize()
            energy.append((uff.CalcEnergy(), cid))
    if ff == 'mmff':
        prop = AllChem.MMFFGetMoleculeProperties(m_h)
        for cid in cids:
            mmff = AllChem.MMFFGetMoleculeForceField(m_h, prop,
                                                     confId=cid)
            mmff.Minimize()
            energy.append((mmff.CalcEnergy(), cid))

    # 4. エネルギーをソートし，相対エネルギーとIDをリストに格納
    energy.sort()
    return [(i-energy[0][0],j) for i,j in energy]


uff_e = opt_sp_mm(mol, 'uff')
mmff_e = opt_sp_mm(mol, 'mmff')

In [3]:
# コンフォマーを発生させ構造最適化を行う関数
#mol="O=C1C(C)(C2(C)(C))CC[C@@H]2C1"
def get_confs(mol, ff):
    #sm = Chem.MolToSmiles(mol)
    m = Chem.MolFromSmiles(mol)
    m_h = Chem.AddHs(m)
    cids = AllChem.EmbedMultipleConfs(m_h,
                                      numConfs=1000,
                                      randomSeed=1234,
                                      pruneRmsThresh=0.1,
                                      numThreads=0)
    if ff == 'uff':
        for cid in cids:
            uff = AllChem.UFFGetMoleculeForceField(m_h,
                                                   confId=cid)
            uff.Minimize()
    if ff == 'mmff':
        prop = AllChem.MMFFGetMoleculeProperties(m_h)
        for cid in cids:
            mmff = AllChem.MMFFGetMoleculeForceField(m_h,
                                                     prop,
                                                     confId=cid)
            mmff.Minimize()
    return m_h#Chem.RemoveHs(m_h)


# 5つの安定構造のIDを取得
uff_confIds = [j for i,j in uff_e[:5]]#[:5]
mmff_confIds = [j for i,j in mmff_e[:5]]#[:5]

# UFF及びMMFFで最適化したコンフォマーを取得
uff = get_confs(mol, 'uff')
mmff = get_confs(mol, 'mmff')

# ベンゼンをテンプレートとして一致部位の取得
core = uff.GetSubstructMatch(Chem.MolFromSmiles('O=CC'))

# テンプレートを用いてコンフォマーの整列
AllChem.AlignMolConformers(uff, atomIds=core)
AllChem.AlignMolConformers(mmff, atomIds=core)

# コンフォマーを重ね合わせて描画
v = py3Dmol.view(width=600, height=600)
print(len(uff_confIds))
for cid in uff_confIds:
    print(cid)
    IPythonConsole.addMolToView(uff, confId=cid, view=v)
#v.setBackgroundColor('')#0xeeeeee
#v.setStyle({'line': {}})
v.zoomTo()
v.show()

5
237
47
46
302
116


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol